In [2]:
import gymnasium as gym
import pandas as pd
import matplotlib.pyplot as plt
import time
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import joblib
import numpy as np
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from env.flo_portfolio import FlorianPortfolioEnv
from env.portfolio_multi_reward import FlorianPortfolioEnvMultiReward
from env.portfolio_bbg import Portfolio_BBG
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import ProgressBarCallback,CallbackList, EvalCallback
from models.callbacks import CustomCallBack, HParamCallback, TensorboardCallback
from models.models import DRLAgent
from gymnasium import spaces
from pyfolio import timeseries
import pyfolio
from plot.plot import convert_daily_return_to_pyfolio_ts, convert_account_value_to_pyfolio_ts, get_baseline, backtest_stats, get_daily_return
from hyperoptimizer.optunaoptimizer import optimize_optuna, optimize_optuna_FlorianPortfolioEnvMultiReward, optimize_optuna_BBG_Env
from configs import tickers
from preprocessors.preprocessors import DataProcessor
# from wandb_env import wandb_opt
import requests
from datetime import datetime, timedelta
import calendar
from eod import EodHistoricalData
from pytz import timezone
import pandas_market_calendars as mcal
from stockstats import StockDataFrame
from preprocessors.eod_downloader import fetch_eod_1d_histroical_API


/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/pyfolio/pos.py:25: UserWarning:

Module "zipline.assets" not found; multipliers will not be applied to position notionals.



In [3]:
ticker = ["UNH.US","JNJ.US","MRK.US","PFE.US", "IBM.US" ]
pharma = ["UNH.US","TMO.US","ABT.US", "LLY.US","NVO.US","SYK.US", "JNJ.US", "MRK.US", "PFE.US", "DHR.US", "AMGN.US", "BMY.US", "MDT.US", "BDX.US", "HUM.US", "WST.US"]
start = "1970-01-01"
end = "2023-08-23"

df = fetch_eod_1d_histroical_API(pharma, start,end)


In [10]:
print(len(pharma))

16


In [24]:
df

,date,open,high,low,close,volume,tic,avgvol_50,sma_10,sma_50,...,sma_200,wma_50,rsi_14,volatility_30,volatility_100,stddev_30,dmi_14,adx_14,macd,atr_14
4174,1986-07-10,99.6246,99.7491,98.0004,1.6566,7177752,MRK.US,6.651710e+06,1.7018,1.5967,...,1.2552,1.6455,49.5228,0.0027,0.0243,0.051175,48.7012,43.8421,0.0230,3.1429
4174,1986-07-10,31.7500,32.0000,31.5000,0.2054,54900,NVO.US,7.598200e+04,0.2101,0.2075,...,0.1895,0.2108,41.1862,0.0000,0.0001,0.004958,16.1337,25.6481,-0.0012,0.7181
4174,1986-07-10,63.5880,65.2452,63.2328,0.9636,12489600,PFE.US,9.946464e+06,1.0078,0.9222,...,0.7903,0.9547,48.2893,0.0026,0.0056,0.051341,9.6559,36.4595,0.0193,1.6110
4174,1986-07-10,40.0016,40.2512,38.7504,0.6693,1627200,SYK.US,8.935680e+05,0.6797,0.6053,...,0.4707,0.6326,56.2838,0.0012,0.0054,0.036175,10.2679,44.1868,0.0197,1.2972
4174,1986-07-10,28.1757,28.9823,27.6917,4.4631,100237,TMO.US,1.241426e+05,4.4767,4.1857,...,3.3849,4.2808,55.3608,0.0424,0.2187,0.20503,22.3155,28.7068,0.0798,0.8396
4174,1986-07-10,12.7504,13.0000,12.7504,0.3227,1670400,UNH.US,2.044288e+06,0.3312,0.3302,...,0.3052,0.3278,47.1105,0.0003,0.0007,0.017224,3.8743,13.5223,-0.0004,0.5090
4174,1986-07-10,28.3752,28.3752,28.3752,1.9682,10400,WST.US,4.699200e+04,2.0402,2.2191,...,1.9542,2.1950,18.9052,0.0191,0.0115,0.136762,66.5070,51.7329,-0.0767,0.3924
4175,1986-07-11,50.3756,51.3754,50.2507,1.2610,4396516,ABT.US,5.963365e+06,1.2877,1.1628,...,0.9190,1.2111,54.7622,0.0043,0.0171,0.066383,25.6266,32.9294,0.0296,2.1779
4175,1986-07-11,22.3728,22.7520,22.1232,0.3461,7593600,AMGN.US,6.937632e+06,0.3583,0.3917,...,0.2547,0.3869,39.2792,0.0008,0.0056,0.027251,30.4065,26.4163,-0.0146,1.0819
4175,1986-07-11,53.8748,54.8752,53.5001,3.7807,1203760,BDX.US,1.204531e+06,3.8722,3.6381,...,2.6823,3.7486,50.7035,0.0134,0.2541,0.113917,3.2675,17.7869,0.0507,1.6023


In [25]:
# Group by date and count unique tickers
date_counts = df.groupby('date')['tic'].nunique()

# Find the total number of unique tickers in the entire DataFrame
total_tickers = df['tic'].nunique()

# Get the first date where the count of unique tickers matches the total tickers
first_common_date = date_counts[date_counts == total_tickers].index.min()

# Filter the DataFrame
filtered_df = df[df['date'] >= first_common_date]

In [43]:
test = filtered_df[filtered_df["date"] > "1986-01-01"]

In [44]:
df = test

In [5]:
df = df.dropna(axis=0)

In [46]:
df = df[df['tic'] != 'WST.US']

In [ ]:
df = df[df['tic'] != 'NVO.US']

In [47]:
df.to_pickle("pharma.pkl")

In [15]:
df.head()

,date,open,high,low,close,volume,tic,avgvol_50,sma_10,sma_50,...,sma_200,wma_50,rsi_14,volatility_30,volatility_100,stddev_30,dmi_14,adx_14,macd,atr_14
199,1985-10-16,45.1248,45.7504,45.0000,1.3096,7308800,JNJ.US,5176192.000,1.2705,1.3043,...,1.2416,1.2923,55.6090,0.0014,0.0013,0.03776,9.0289,22.0343,-0.0058,0.9246
199,1985-10-16,109.6262,112.9991,109.5017,0.9218,12499286,MRK.US,6432473.060,0.8896,0.9094,...,0.8542,0.9012,63.0910,0.0003,0.0005,0.018639,13.7934,30.7188,-0.0023,1.4162
199,1985-10-16,43.5756,43.5756,42.6288,0.6604,15031200,PFE.US,9893423.713,0.6580,0.6772,...,0.6500,0.6693,47.2648,0.0003,0.0007,0.016981,26.4289,18.7649,-0.0065,1.1282
199,1985-10-16,10.3744,10.5008,10.0000,0.2626,1926400,UNH.US,1049280.000,0.2468,0.2439,...,0.2270,0.2389,60.7769,0.0003,0.0007,0.018692,26.9431,26.8106,0.0049,0.5354
200,1985-10-17,45.6256,46.8752,45.6256,1.3348,10320000,JNJ.US,5170496.000,1.2763,1.3050,...,1.2431,1.2935,60.2921,0.0013,0.0013,0.037623,25.3820,22.2734,-0.0010,0.9478


In [2]:
def get_intraday_eod(tickers, start, end, interval_len):
    client = EodHistoricalData("64cbaf10539312.00750558")
    data_df = pd.DataFrame()
    for ticker in tickers:
        try:
            resp = client.get_prices_intraday(ticker, interval=interval_len, from_=start, to=end)
            temp_df = pd.DataFrame(resp)
            if len(temp_df) > 0:
                # temp_df = temp_df.drop(columns=['timestamp','gmtoffset'])
                temp_df['ticker'] = ticker
                data_df = pd.concat([data_df, temp_df], axis=0)
            else:
                print(f"Dataframe empty for ticker: {ticker}")
        except Exception as e:
            print(f"Error fetching data for ticker {ticker}: {e}")
    return data_df


In [3]:
def fetch_eod_data():
    tickers = ['AAPL.US',"MSFT.US","AMZN.US"]
    start_date = '2021-01-01'
    end_date = '2023-08-14'
    # Convert them to datetime objects
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    # Initialize the start and end dates for the first 120-day period
    period_start_date = start_date
    period_end_date = start_date + timedelta(days=100) # Subtract one to avoid overlap with next period
    data_df = pd.DataFrame()
    # Loop over 120-day periods
    while period_end_date <= end_date:
        # Convert dates to UNIX timestamp
        period_start_unix = int(time.mktime(period_start_date.timetuple()))
        period_end_unix = int(time.mktime(period_end_date.timetuple()))

        # Fetch and process data for this period
            

        try:
            temp_df = get_intraday_eod(tickers=tickers, start=period_start_unix, end=period_end_unix, interval_len="1m")
            if len(temp_df) > 0:
                data_df = pd.concat([data_df, temp_df], axis=0)
                data_df.to_csv("data_df_while_loading.csv")
            else:
                print(f"Dataframe empty for ticker: {period_start_unix}")
        except Exception as e:
            print(f"Error fetching data for ticker {period_start_unix}: {e}")
        # Update the start and end dates for the next 120-day period
        period_start_date = period_end_date + timedelta(days=1)
        period_end_date = period_start_date + timedelta(days=100)
    return data_df

In [10]:
df = fetch_eod_data()

In [11]:
df.to_pickle("tech_stock_try.pkl")

In [29]:
df = pd.read_pickle("tech_stock_try.pkl")
df

,timestamp,gmtoffset,datetime,open,high,low,close,volume,ticker
0,1609455600,0,2020-12-31 23:00:00,132.6000,132.60,132.57,132.57,387,AAPL.US
1,1609455660,0,2020-12-31 23:01:00,132.6000,132.60,132.60,132.60,646,AAPL.US
2,1609455720,0,2020-12-31 23:02:00,132.5800,132.58,132.58,132.58,697,AAPL.US
3,1609455780,0,2020-12-31 23:03:00,132.5700,132.60,132.57,132.60,656,AAPL.US
4,1609455840,0,2020-12-31 23:04:00,132.5800,132.58,132.58,132.58,258,AAPL.US
...,...,...,...,...,...,...,...,...,...
53553,1687902960,0,2023-06-27 21:56:00,129.1200,129.14,129.10,129.12,72,AMZN.US
53554,1687903020,0,2023-06-27 21:57:00,129.1400,129.14,129.12,129.12,11,AMZN.US
53555,1687903080,0,2023-06-27 21:58:00,129.1200,129.14,129.12,129.14,14,AMZN.US
53556,1687903140,0,2023-06-27 21:59:00,129.1200,129.14,129.12,129.14,23,AMZN.US


In [30]:
df = df.drop(columns=['timestamp', "gmtoffset"])
df['datetime'] = pd.to_datetime(df['datetime'])

df['datetime'] = df['datetime'].dt.tz_localize('UTC')


df['datetime'] = df['datetime'].dt.tz_convert('America/New_York')

df = df.set_index('datetime')
df = df.between_time('9:30', '16:00')
df

,open,high,low,close,volume,ticker
datetime,,,,,,
2021-01-04 09:30:00-05:00,133.520,133.6116,132.95,133.1500,2455841,AAPL.US
2021-01-04 09:31:00-05:00,133.130,133.4500,133.08,133.3350,534463,AAPL.US
2021-01-04 09:32:00-05:00,133.345,133.3600,132.99,133.1100,526051,AAPL.US
2021-01-04 09:33:00-05:00,133.110,133.1500,132.71,132.7462,555529,AAPL.US
2021-01-04 09:34:00-05:00,132.760,132.8300,132.39,132.8100,744788,AAPL.US
...,...,...,...,...,...,...
2023-06-27 15:56:00-04:00,129.096,129.1200,129.01,129.0300,169739,AMZN.US
2023-06-27 15:57:00-04:00,129.030,129.1950,129.03,129.1510,484542,AMZN.US
2023-06-27 15:58:00-04:00,129.150,129.1600,129.09,129.1350,221252,AMZN.US


In [31]:
df = df.groupby('ticker').resample('D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})


In [32]:
df = df.reset_index()
df

,ticker,datetime,open,high,low,close,volume
0,AAPL.US,2021-01-04 00:00:00-05:00,133.52,133.6116,126.76,129.56,121867785
1,AAPL.US,2021-01-05 00:00:00-05:00,128.89,131.7400,128.43,131.08,84495497
2,AAPL.US,2021-01-06 00:00:00-05:00,127.72,131.0499,126.36,126.40,139627193
3,AAPL.US,2021-01-07 00:00:00-05:00,128.36,131.6300,127.86,130.86,94170603
4,AAPL.US,2021-01-08 00:00:00-05:00,132.43,132.6300,130.23,132.06,92475151
...,...,...,...,...,...,...,...
2710,MSFT.US,2023-06-23 00:00:00-04:00,334.36,337.9600,333.45,335.02,25981898
2711,MSFT.US,2023-06-24 00:00:00-04:00,NaN,NaN,NaN,NaN,0
2712,MSFT.US,2023-06-25 00:00:00-04:00,NaN,NaN,NaN,NaN,0
2713,MSFT.US,2023-06-26 00:00:00-04:00,333.72,336.1100,328.44,328.84,23208365


In [34]:

df

,ticker,datetime,open,high,low,close,volume
0,AAPL.US,2021-01-04 00:00:00-05:00,133.52,133.6116,126.76,129.56,121867785
1,AAPL.US,2021-01-05 00:00:00-05:00,128.89,131.7400,128.43,131.08,84495497
2,AAPL.US,2021-01-06 00:00:00-05:00,127.72,131.0499,126.36,126.40,139627193
3,AAPL.US,2021-01-07 00:00:00-05:00,128.36,131.6300,127.86,130.86,94170603
4,AAPL.US,2021-01-08 00:00:00-05:00,132.43,132.6300,130.23,132.06,92475151
...,...,...,...,...,...,...,...
2710,MSFT.US,2023-06-23 00:00:00-04:00,334.36,337.9600,333.45,335.02,25981898
2711,MSFT.US,2023-06-24 00:00:00-04:00,NaN,NaN,NaN,NaN,0
2712,MSFT.US,2023-06-25 00:00:00-04:00,NaN,NaN,NaN,NaN,0
2713,MSFT.US,2023-06-26 00:00:00-04:00,333.72,336.1100,328.44,328.84,23208365


In [35]:
df = df.rename(columns={'datetime': 'date',"ticker": "tic"})
df

,tic,date,open,high,low,close,volume
0,AAPL.US,2021-01-04 00:00:00-05:00,133.52,133.6116,126.76,129.56,121867785
1,AAPL.US,2021-01-05 00:00:00-05:00,128.89,131.7400,128.43,131.08,84495497
2,AAPL.US,2021-01-06 00:00:00-05:00,127.72,131.0499,126.36,126.40,139627193
3,AAPL.US,2021-01-07 00:00:00-05:00,128.36,131.6300,127.86,130.86,94170603
4,AAPL.US,2021-01-08 00:00:00-05:00,132.43,132.6300,130.23,132.06,92475151
...,...,...,...,...,...,...,...
2710,MSFT.US,2023-06-23 00:00:00-04:00,334.36,337.9600,333.45,335.02,25981898
2711,MSFT.US,2023-06-24 00:00:00-04:00,NaN,NaN,NaN,NaN,0
2712,MSFT.US,2023-06-25 00:00:00-04:00,NaN,NaN,NaN,NaN,0
2713,MSFT.US,2023-06-26 00:00:00-04:00,333.72,336.1100,328.44,328.84,23208365


In [36]:
df = df.sort_values(['date','tic'],ignore_index=True)


df.index = df["date"].factorize()[0]


In [37]:
df

,tic,date,open,high,low,close,volume
0,AAPL.US,2021-01-04 00:00:00-05:00,133.52,133.6116,126.76,129.56,121867785
0,AMZN.US,2021-01-04 00:00:00-05:00,3270.00,3272.0000,3144.02,3186.63,3909734
0,MSFT.US,2021-01-04 00:00:00-05:00,222.53,223.0000,214.81,217.77,27384646
1,AAPL.US,2021-01-05 00:00:00-05:00,128.89,131.7400,128.43,131.08,84495497
1,AMZN.US,2021-01-05 00:00:00-05:00,3166.01,3223.3800,3165.06,3218.51,2311620
...,...,...,...,...,...,...,...
903,AMZN.US,2023-06-26 00:00:00-04:00,129.33,131.4900,127.10,127.51,65450273
903,MSFT.US,2023-06-26 00:00:00-04:00,333.72,336.1100,328.44,328.84,23208365
904,AAPL.US,2023-06-27 00:00:00-04:00,185.89,188.3900,185.67,188.04,54255382
904,AMZN.US,2023-06-27 00:00:00-04:00,128.63,130.0850,127.55,129.18,49715605


In [38]:
df = df.dropna(axis=0)
df

,tic,date,open,high,low,close,volume
0,AAPL.US,2021-01-04 00:00:00-05:00,133.52,133.6116,126.76,129.56,121867785
0,AMZN.US,2021-01-04 00:00:00-05:00,3270.00,3272.0000,3144.02,3186.63,3909734
0,MSFT.US,2021-01-04 00:00:00-05:00,222.53,223.0000,214.81,217.77,27384646
1,AAPL.US,2021-01-05 00:00:00-05:00,128.89,131.7400,128.43,131.08,84495497
1,AMZN.US,2021-01-05 00:00:00-05:00,3166.01,3223.3800,3165.06,3218.51,2311620
...,...,...,...,...,...,...,...
903,AMZN.US,2023-06-26 00:00:00-04:00,129.33,131.4900,127.10,127.51,65450273
903,MSFT.US,2023-06-26 00:00:00-04:00,333.72,336.1100,328.44,328.84,23208365
904,AAPL.US,2023-06-27 00:00:00-04:00,185.89,188.3900,185.67,188.04,54255382
904,AMZN.US,2023-06-27 00:00:00-04:00,128.63,130.0850,127.55,129.18,49715605


In [39]:

rsi_period = 14
sma_period = 60
macd_fastperiod = 12
macd_slowperiod = 26
macd_signalperiod = 9

tickers = df['tic'].unique()


result_df = pd.DataFrame()

for ticker in tickers:
    ticker_df = df[df['tic'] == ticker].copy()


    delta = ticker_df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    average_gain = gain.rolling(window=rsi_period).mean()
    average_loss = loss.rolling(window=rsi_period).mean()
    rs = average_gain / average_loss
    rsi = 100 - (100 / (1 + rs))
    ticker_df['rsi'] = rsi


    ticker_df['sma'] = ticker_df['close'].rolling(window=sma_period).mean()


    ema_fast = ticker_df['close'].ewm(span=macd_fastperiod).mean()
    ema_slow = ticker_df['close'].ewm(span=macd_slowperiod).mean()
    ticker_df['macd'] = ema_fast - ema_slow
    ticker_df['macds'] = ticker_df['macd'].ewm(span=macd_signalperiod).mean()
    ticker_df['macdh'] = ticker_df['macd'] - ticker_df['macds']


    result_df = pd.concat([result_df, ticker_df], ignore_index=True)


# result_df = result_df.set_index(['date'])


result_df = result_df.dropna(axis=0)

df = result_df.copy()
df

,tic,date,open,high,low,close,volume,rsi,sma,macd,macds,macdh
59,AAPL.US,2021-03-30 00:00:00-04:00,120.11,120.4031,118.86,119.80,76251668,49.709951,128.461083,-1.690304,-1.899389,0.209085
60,AAPL.US,2021-03-31 00:00:00-04:00,121.65,123.5200,121.15,122.15,95741678,50.548486,128.337583,-1.508233,-1.821158,0.312925
61,AAPL.US,2021-04-01 00:00:00-04:00,123.66,124.1800,122.49,122.96,66042395,54.245491,128.202250,-1.284285,-1.713783,0.429499
62,AAPL.US,2021-04-05 00:00:00-04:00,123.87,126.1601,123.07,125.90,79444725,54.164830,128.193917,-0.861370,-1.543301,0.681930
63,AAPL.US,2021-04-06 00:00:00-04:00,126.50,127.1300,125.65,126.10,71654993,51.150775,128.114583,-0.504384,-1.335517,0.831133
...,...,...,...,...,...,...,...,...,...,...,...,...
1852,MSFT.US,2023-06-21 00:00:00-04:00,336.37,337.7300,332.07,333.71,18150046,54.437064,308.130000,7.363925,7.869198,-0.505273
1853,MSFT.US,2023-06-22 00:00:00-04:00,334.12,340.1200,333.34,339.73,24881935,55.909016,309.185833,7.220130,7.739385,-0.519254
1854,MSFT.US,2023-06-23 00:00:00-04:00,334.36,337.9600,333.45,335.02,25981898,49.631764,310.183667,6.649463,7.521400,-0.871937
1855,MSFT.US,2023-06-26 00:00:00-04:00,333.72,336.1100,328.44,328.84,23208365,44.809384,310.989500,5.633591,7.143838,-1.510248


In [40]:


df.index = df["date"].factorize()[0]
# df = df.sort_values(['date','tic'],ignore_index=True)
df

,tic,date,open,high,low,close,volume,rsi,sma,macd,macds,macdh
0,AAPL.US,2021-03-30 00:00:00-04:00,120.11,120.4031,118.86,119.80,76251668,49.709951,128.461083,-1.690304,-1.899389,0.209085
1,AAPL.US,2021-03-31 00:00:00-04:00,121.65,123.5200,121.15,122.15,95741678,50.548486,128.337583,-1.508233,-1.821158,0.312925
2,AAPL.US,2021-04-01 00:00:00-04:00,123.66,124.1800,122.49,122.96,66042395,54.245491,128.202250,-1.284285,-1.713783,0.429499
3,AAPL.US,2021-04-05 00:00:00-04:00,123.87,126.1601,123.07,125.90,79444725,54.164830,128.193917,-0.861370,-1.543301,0.681930
4,AAPL.US,2021-04-06 00:00:00-04:00,126.50,127.1300,125.65,126.10,71654993,51.150775,128.114583,-0.504384,-1.335517,0.831133
...,...,...,...,...,...,...,...,...,...,...,...,...
555,MSFT.US,2023-06-21 00:00:00-04:00,336.37,337.7300,332.07,333.71,18150046,54.437064,308.130000,7.363925,7.869198,-0.505273
556,MSFT.US,2023-06-22 00:00:00-04:00,334.12,340.1200,333.34,339.73,24881935,55.909016,309.185833,7.220130,7.739385,-0.519254
557,MSFT.US,2023-06-23 00:00:00-04:00,334.36,337.9600,333.45,335.02,25981898,49.631764,310.183667,6.649463,7.521400,-0.871937
558,MSFT.US,2023-06-26 00:00:00-04:00,333.72,336.1100,328.44,328.84,23208365,44.809384,310.989500,5.633591,7.143838,-1.510248


In [42]:
df = df.sort_values(by=["date","tic"])
df

,tic,date,open,high,low,close,volume,rsi,sma,macd,macds,macdh
0,AAPL.US,2021-03-30 00:00:00-04:00,120.110,120.4031,118.86,119.80,76251668,49.709951,128.461083,-1.690304,-1.899389,0.209085
0,AMZN.US,2021-03-30 00:00:00-04:00,3070.010,3073.0000,3034.00,3059.99,2087539,50.237503,3171.706667,-22.922129,-27.411558,4.489429
0,MSFT.US,2021-03-30 00:00:00-04:00,233.525,233.8500,231.10,232.09,20593233,49.986762,232.029172,-0.102565,-0.066862,-0.035704
1,AAPL.US,2021-03-31 00:00:00-04:00,121.650,123.5200,121.15,122.15,95741678,50.548486,128.337583,-1.508233,-1.821158,0.312925
1,AMZN.US,2021-03-31 00:00:00-04:00,3064.060,3119.3300,3062.50,3094.08,2551818,47.937065,3170.164167,-19.799901,-25.889225,6.089323
...,...,...,...,...,...,...,...,...,...,...,...,...
558,AMZN.US,2023-06-26 00:00:00-04:00,129.330,131.4900,127.10,127.51,65450273,54.192790,112.828293,3.736479,4.007008,-0.270530
558,MSFT.US,2023-06-26 00:00:00-04:00,333.720,336.1100,328.44,328.84,23208365,44.809384,310.989500,5.633591,7.143838,-1.510248
559,AAPL.US,2023-06-27 00:00:00-04:00,185.890,188.3900,185.67,188.04,54255382,72.163918,173.240000,3.555371,3.517194,0.038177
559,AMZN.US,2023-06-27 00:00:00-04:00,128.630,130.0850,127.55,129.18,49715605,55.003849,113.281277,3.691072,3.943821,-0.252749


In [43]:
df.to_pickle("tec_test_done.pkl")

In [8]:
df = df.drop(columns=['timestamp', "gmtoffset"])
df['datetime'] = pd.to_datetime(df['datetime'])

df['datetime'] = df['datetime'].dt.tz_localize('UTC')


df['datetime'] = df['datetime'].dt.tz_convert('America/New_York')

df = df.set_index('datetime')
df = df.between_time('9:30', '16:00')


df.index = df.index - pd.Timedelta(minutes=30)


df = df.groupby('ticker').resample('D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})
# df = df.groupby('ticker').resample('60T').agg({
#     'open': 'first',
#     'high': 'max',
#     'low': 'min',
#     'close': 'last',
#     'volume': 'sum'
# })


df = df.reset_index()
df['datetime'] = df['datetime'] + pd.Timedelta(minutes=30)


df = df.set_index('datetime')
df = df.between_time('9:30', '16:00').reset_index()

nyse = mcal.get_calendar('NYSE')


# schedule = nyse.schedule(start_date=df['datetime'].min(), end_date=df['datetime'].max())

# Compare the date part of 'datetime' in df with the date in schedule
# df = df[df['datetime'].dt.date.isin(schedule.index.date)]

df


KeyError: "['timestamp', 'gmtoffset'] not found in axis"

In [9]:
df.isna().any()


,datetime,ticker,open,high,low,close,volume
0,2003-09-10 09:30:00-04:00,SPY.US,102.5300,102.800,102.2470,102.3200,12410900.0
1,2003-09-10 10:30:00-04:00,SPY.US,102.3400,102.550,102.3000,102.3500,3270700.0
2,2003-09-10 11:30:00-04:00,SPY.US,102.3500,102.440,102.1500,102.3600,4127100.0
3,2003-09-10 12:30:00-04:00,SPY.US,102.3500,102.600,102.2900,102.3500,2411300.0
4,2003-09-10 13:30:00-04:00,SPY.US,102.3500,102.410,101.8600,101.8900,6428400.0
...,...,...,...,...,...,...,...
50864,2023-08-02 11:30:00-04:00,SPY.US,451.1200,451.670,449.9800,450.2396,10400750.0
50865,2023-08-02 12:30:00-04:00,SPY.US,450.2300,451.200,450.0750,451.1100,6467510.0
50866,2023-08-02 13:30:00-04:00,SPY.US,451.1100,451.435,450.4550,450.7950,5584619.0
50867,2023-08-02 14:30:00-04:00,SPY.US,450.7800,450.925,449.3800,449.4100,8124297.0


In [11]:
df = df.rename(columns={'datetime': 'date',"ticker": "tic"})


df = df.groupby('tic').apply(lambda group: group.ffill()).reset_index(drop=True)


df = df.sort_values(['date','tic'],ignore_index=True)


df.index = df["date"].factorize()[0]

df = df.dropna(axis=0)
df.to_pickle("SPY_since2003_1h.pkl")

In [12]:

rsi_period = 91
sma_period = 325
macd_fastperiod = 78
macd_slowperiod = 169
macd_signalperiod = 59

tickers = df['tic'].unique()


result_df = pd.DataFrame()

for ticker in tickers:
    ticker_df = df[df['tic'] == ticker].copy()


    delta = ticker_df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    average_gain = gain.rolling(window=rsi_period).mean()
    average_loss = loss.rolling(window=rsi_period).mean()
    rs = average_gain / average_loss
    rsi = 100 - (100 / (1 + rs))
    ticker_df['rsi'] = rsi


    ticker_df['sma'] = ticker_df['close'].rolling(window=sma_period).mean()


    ema_fast = ticker_df['close'].ewm(span=macd_fastperiod).mean()
    ema_slow = ticker_df['close'].ewm(span=macd_slowperiod).mean()
    ticker_df['macd'] = ema_fast - ema_slow
    ticker_df['macds'] = ticker_df['macd'].ewm(span=macd_signalperiod).mean()
    ticker_df['macdh'] = ticker_df['macd'] - ticker_df['macds']


    result_df = pd.concat([result_df, ticker_df], ignore_index=True)


result_df = result_df.set_index(['date'])


result_df = result_df.dropna(axis=0)

df = result_df.copy()
df

,,open,high,low,close,volume,rsi,sma,macd,macds,macdh
date,tic,,,,,,,,,,
2003-11-13 11:30:00-05:00,SPY.US,105.9700,106.140,105.9000,106.1000,3740300.0,56.923077,103.783797,0.513280,0.508102,0.005179
2003-11-13 12:30:00-05:00,SPY.US,106.0900,106.250,106.0000,106.0600,2428200.0,57.159834,103.795305,0.515441,0.508346,0.007095
2003-11-13 13:30:00-05:00,SPY.US,106.0500,106.160,105.8900,106.0500,1463700.0,57.708458,103.806689,0.517235,0.508642,0.008592
2003-11-13 14:30:00-05:00,SPY.US,106.0400,106.540,105.9600,106.1800,5911700.0,58.070280,103.818443,0.520538,0.509039,0.011499
2003-11-13 15:30:00-05:00,SPY.US,106.1800,106.500,106.1600,106.3700,2336200.0,58.608491,103.830812,0.526094,0.509608,0.016487
...,...,...,...,...,...,...,...,...,...,...,...
2023-08-02 11:30:00-04:00,SPY.US,451.1200,451.670,449.9800,450.2396,10400750.0,50.383808,439.731068,4.950349,5.476255,-0.525906
2023-08-02 12:30:00-04:00,SPY.US,450.2300,451.200,450.0750,451.1100,6467510.0,51.062566,439.847929,4.855381,5.455559,-0.600178
2023-08-02 13:30:00-04:00,SPY.US,451.1100,451.435,450.4550,450.7950,5584619.0,50.939190,439.962375,4.758192,5.432313,-0.674122


In [13]:
df.to_pickle("SPY_since2003_1h.pkl")

In [ ]:
result_df = result_df.set_index(['date'])

In [2]:
tickers = ["MSFT.US", "AMZN.US", "AAPL.US", "IBM.US", "F.US"]
start_date = "2000-01-05"
end_date = "2023-08-10"
def get_daily_eod(tickers, start, end):
    client = EodHistoricalData("64cbaf10539312.00750558")
    data_df = pd.DataFrame()
    for ticker in tickers:
        try:
            resp = client.get_prices_eod(ticker, period="d", from_=start_date, to=end_date)
            temp_df = pd.DataFrame(resp)
            if len(temp_df) > 0:
                # temp_df = temp_df.drop(columns=['timestamp','gmtoffset'])
                temp_df['ticker'] = ticker
                data_df = pd.concat([data_df, temp_df], axis=0)
            else:
                print(f"Dataframe empty for ticker: {ticker}")
        except Exception as e:
            print(f"Error fetching data for ticker {ticker}: {e}")
    return data_df
daily_data = get_daily_eod(tickers,start_date,end_date)

In [6]:
daily_data.to_csv("temp_data.csv")

In [7]:
daily_data

,date,open,high,low,close,adjusted_close,volume,ticker
0,2000-01-05,111.1250,116.3750,109.3750,113.8125,35.3805,64047000,MSFT.US
1,2000-01-06,112.1875,113.8750,108.3750,110.0000,34.1953,54966602,MSFT.US
2,2000-01-07,108.6250,112.2500,107.3125,111.4375,34.6422,62013398,MSFT.US
3,2000-01-10,113.4375,113.6875,111.3750,112.2500,34.8948,44955200,MSFT.US
4,2000-01-11,111.5000,114.2500,108.6875,109.3750,34.0010,46728000,MSFT.US
...,...,...,...,...,...,...,...,...
5932,2023-08-04,13.1000,13.1000,12.8000,12.8900,12.8900,48325000,F.US
5933,2023-08-07,12.9300,12.9600,12.7800,12.9400,12.9400,43929200,F.US
5934,2023-08-08,12.7700,12.9000,12.7000,12.8800,12.8800,45103800,F.US
5935,2023-08-09,12.7700,12.8300,12.6600,12.7300,12.7300,42932600,F.US


In [9]:
daily_data = daily_data.drop(columns=["close"])
daily_data = daily_data.rename(columns={'adjusted_close': 'close',"ticker": "tic"})
daily_data

,date,open,high,low,close,volume,tic
0,2000-01-05,111.1250,116.3750,109.3750,35.3805,64047000,MSFT.US
1,2000-01-06,112.1875,113.8750,108.3750,34.1953,54966602,MSFT.US
2,2000-01-07,108.6250,112.2500,107.3125,34.6422,62013398,MSFT.US
3,2000-01-10,113.4375,113.6875,111.3750,34.8948,44955200,MSFT.US
4,2000-01-11,111.5000,114.2500,108.6875,34.0010,46728000,MSFT.US
...,...,...,...,...,...,...,...
5932,2023-08-04,13.1000,13.1000,12.8000,12.8900,48325000,F.US
5933,2023-08-07,12.9300,12.9600,12.7800,12.9400,43929200,F.US
5934,2023-08-08,12.7700,12.9000,12.7000,12.8800,45103800,F.US
5935,2023-08-09,12.7700,12.8300,12.6600,12.7300,42932600,F.US


In [10]:
df = daily_data

In [12]:
rsi_period = 14
sma_period = 60
macd_fastperiod = 12
macd_slowperiod = 26
macd_signalperiod = 9

tickers = df['tic'].unique()


result_df = pd.DataFrame()

for ticker in tickers:
    ticker_df = df[df['tic'] == ticker].copy()


    delta = ticker_df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    average_gain = gain.rolling(window=rsi_period).mean()
    average_loss = loss.rolling(window=rsi_period).mean()
    rs = average_gain / average_loss
    rsi = 100 - (100 / (1 + rs))
    ticker_df['rsi'] = rsi


    ticker_df['sma'] = ticker_df['close'].rolling(window=sma_period).mean()


    ema_fast = ticker_df['close'].ewm(span=macd_fastperiod).mean()
    ema_slow = ticker_df['close'].ewm(span=macd_slowperiod).mean()
    ticker_df['macd'] = ema_fast - ema_slow
    ticker_df['macds'] = ticker_df['macd'].ewm(span=macd_signalperiod).mean()
    ticker_df['macdh'] = ticker_df['macd'] - ticker_df['macds']


    result_df = pd.concat([result_df, ticker_df], ignore_index=True)


# result_df = result_df.set_index(['date'])



result_df = result_df.dropna(axis=0)
df = result_df.copy()
df = df.sort_values(["date","tic"])
df.index = df["date"].factorize()[0]
df

,date,open,high,low,close,volume,tic,rsi,sma,macd,macds,macdh
0,2000-03-30,133.5600,137.6928,125.4400,0.9530,414400000,AAPL.US,50.000000,0.872215,0.037923,0.038520,-0.000597
0,2000-03-30,64.5000,67.3750,64.0000,3.3250,100667999,AMZN.US,49.443062,3.421678,0.012873,0.003096,0.009777
0,2000-03-30,45.7501,46.6251,45.1874,12.4226,9839603,F.US,68.016824,12.393152,0.061290,-0.054470,0.115760
0,2000-03-30,118.8750,123.0000,118.6875,65.4073,9579268,IBM.US,73.972575,60.540583,1.689982,0.840771,0.849211
0,2000-03-30,106.1875,108.6250,102.5000,32.1358,64092800,MSFT.US,52.870060,31.545813,0.708178,0.477216,0.230961
...,...,...,...,...,...,...,...,...,...,...,...,...
5877,2023-08-10,179.4800,180.7500,177.6000,177.7300,54686900,AAPL.US,24.553679,184.914965,-2.241844,0.032329,-2.274173
5877,2023-08-10,139.0700,140.4100,137.4900,138.5600,58928400,AMZN.US,63.128834,127.411667,2.753727,2.095961,0.657767
5877,2023-08-10,12.7800,12.7800,12.0800,12.1600,104063900,F.US,22.260274,13.415290,-0.382705,-0.239448,-0.143257
5877,2023-08-10,143.0400,144.5800,142.6900,143.2500,4735800,IBM.US,74.462430,133.478595,2.898735,2.818371,0.080365


In [13]:
df.to_pickle("new_daily_data.pkl")